<a href="https://colab.research.google.com/github/rajivsam/example_ITSM/blob/main/notebooks/colab_version_prepare_raw_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/rajivsam/example_ITSM.git

fatal: destination path 'example_ITSM' already exists and is not an empty directory.


In [2]:
import os
os.chdir('example_ITSM/notebooks')

In [3]:
! pip install -r ../requirements.txt

In [4]:
!pwd

/content/example_ITSM/notebooks


In [5]:
import pandas as pd
fp = "../data/incident_event_log_02.csv"
df = pd.read_csv(fp)

<ipython-input-5-703648aa9f1d>:3: DtypeWarning: Columns (19,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fp)


In [6]:
df_size = pd.DataFrame.from_records([{"rows": df.shape[0], "cols": df.shape[1]}])

In [7]:
col_df = pd.DataFrame({"columns": df.columns.tolist()})

In [8]:
df_nv = df.isnull().any().to_frame().reset_index()
df_nv.columns = ["column", "null_values"]
df_nv.to_json()

'{"column":{"0":"number","1":"incident_state","2":"active","3":"reassignment_count","4":"reopen_count","5":"sys_mod_count","6":"made_sla","7":"caller_id","8":"opened_by","9":"opened_at","10":"sys_created_by","11":"sys_created_at","12":"sys_updated_by","13":"sys_updated_at","14":"contact_type","15":"location","16":"category","17":"subcategory","18":"u_symptom","19":"cmdb_ci","20":"impact","21":"urgency","22":"priority","23":"assignment_group","24":"assigned_to","25":"knowledge","26":"u_priority_confirmation","27":"notify","28":"problem_id","29":"rfc","30":"vendor","31":"caused_by","32":"closed_code","33":"resolved_by","34":"resolved_at","35":"closed_at"},"null_values":{"0":false,"1":false,"2":false,"3":false,"4":false,"5":false,"6":false,"7":true,"8":true,"9":false,"10":true,"11":true,"12":false,"13":false,"14":false,"15":true,"16":true,"17":true,"18":true,"19":true,"20":false,"21":false,"22":false,"23":true,"24":true,"25":false,"26":false,"27":false,"28":true,"29":true,"30":true,"31":t

In [9]:
df_raw_types = df.dtypes.to_frame().reset_index()
df_raw_types.columns = ["column", "dtype"]
df_raw_types["dtype"] = df_raw_types["dtype"].astype(str)
df_raw_summary = df_raw_types.merge(df_nv, on="column")
df_raw_summary

,column,dtype,null_values
0,number,object,False
1,incident_state,object,False
2,active,bool,False
3,reassignment_count,int64,False
4,reopen_count,int64,False
5,sys_mod_count,int64,False
6,made_sla,bool,False
7,caller_id,object,True
8,opened_by,object,True
9,opened_at,object,False


In [10]:
cols = df.columns.tolist()
int_cols = [c for c in cols if "count" in c]
date_time_cols = [c for c in cols if "_at" in c]


In [11]:
raw_type_info = df.dtypes.to_dict()
mod_type_info = raw_type_info.copy()

In [12]:
import dateutil.parser as parser

def valid_time_stamp(x):
    try:
        pdt = parser.parse(x)
        return True
    except:
        return False

In [13]:
from dask.distributed import Client
dask_client = Client()
dask_client

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:35749
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:39861'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:43489'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:37089', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:37089
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:39216
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:34623', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:34623
INFO:distributed.core:Sta

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 12.68 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35749,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 12.68 GiB
Comm: tcp://127.0.0.1:34623,Total threads: 1
Dashboard: http://127.0.0.1:40943/status,Memory: 6.34 GiB
Nanny: tcp://127.0.0.1:39861,


In [14]:
import dask
from dask.dataframe import from_pandas
ddf = from_pandas(df, npartitions=4)

In [15]:
valid_sys_creation = ddf["sys_created_at"].apply(valid_time_stamp, meta=('sys_created_at', str)).compute()

/usr/local/lib/python3.10/dist-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 28.70 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [16]:
valid_opened_at = ddf["opened_at"].apply(valid_time_stamp, meta=('opened_at', str)).compute()

/usr/local/lib/python3.10/dist-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 28.70 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [17]:
valid_resolved_at = ddf["resolved_at"].apply(valid_time_stamp, meta=('resolved_at', str)).compute()

/usr/local/lib/python3.10/dist-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 28.70 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [18]:
valid_closed_at = ddf["closed_at"].apply(valid_time_stamp, meta=('closed_at', str)).compute()

/usr/local/lib/python3.10/dist-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 28.70 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [19]:
valid_dates = valid_sys_creation & valid_closed_at & valid_opened_at & valid_resolved_at

In [20]:
ddf = ddf[valid_dates].compute()

/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:4880: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  meta = self._meta[_extract_meta(key)]
/usr/local/lib/python3.10/dist-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 28.84 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [21]:
ddf.shape

(88628, 36)

In [22]:
import dask as dd
ddf["sys_created_at"] = ddf["sys_created_at"].apply(lambda x: parser.parse(x))
ddf["opened_at"] = ddf["opened_at"].apply(lambda x: parser.parse(x))
ddf["resolved_at"] = ddf["resolved_at"].apply(lambda x: parser.parse(x))
ddf["closed_at"] = ddf["closed_at"].apply(lambda x: parser.parse(x))
ddf["sys_updated_at"] = ddf["sys_updated_at"].apply(lambda x: parser.parse(x))

In [23]:
for k,v in mod_type_info.items():
    if k in int_cols:
        mod_type_info[k] = "int64"
    if k in date_time_cols:
        mod_type_info[k] = "datetime64[ns]"


In [24]:
category_cols = [ c for c in cols if (c not in int_cols and c not in date_time_cols)]

In [25]:
category_cols

['number',
 'incident_state',
 'active',
 'made_sla',
 'caller_id',
 'opened_by',
 'sys_created_by',
 'sys_updated_by',
 'contact_type',
 'location',
 'category',
 'subcategory',
 'u_symptom',
 'cmdb_ci',
 'impact',
 'urgency',
 'priority',
 'assignment_group',
 'assigned_to',
 'knowledge',
 'u_priority_confirmation',
 'notify',
 'problem_id',
 'rfc',
 'vendor',
 'caused_by',
 'closed_code',
 'resolved_by']

In [26]:
nan_strings = "UNKNOWN"
for c in category_cols:
    ddf[c] = ddf[c].fillna("UNKNOWN")
    ddf[c] = ddf[c].astype(str)

In [27]:
df_nv_fix_check = ddf.isnull().any().to_frame().reset_index()
df_nv_fix_check.columns = ["columns", "null_values"]

In [28]:
df_nv_fix_check

,columns,null_values
0,number,False
1,incident_state,False
2,active,False
3,reassignment_count,False
4,reopen_count,False
5,sys_mod_count,False
6,made_sla,False
7,caller_id,False
8,opened_by,False
9,opened_at,False


In [29]:
mod_type_info

{'number': dtype('O'),
 'incident_state': dtype('O'),
 'active': dtype('bool'),
 'reassignment_count': 'int64',
 'reopen_count': 'int64',
 'sys_mod_count': 'int64',
 'made_sla': dtype('bool'),
 'caller_id': dtype('O'),
 'opened_by': dtype('O'),
 'opened_at': 'datetime64[ns]',
 'sys_created_by': dtype('O'),
 'sys_created_at': 'datetime64[ns]',
 'sys_updated_by': dtype('O'),
 'sys_updated_at': 'datetime64[ns]',
 'contact_type': dtype('O'),
 'location': dtype('O'),
 'category': dtype('O'),
 'subcategory': dtype('O'),
 'u_symptom': dtype('O'),
 'cmdb_ci': dtype('O'),
 'impact': dtype('O'),
 'urgency': dtype('O'),
 'priority': dtype('O'),
 'assignment_group': dtype('O'),
 'assigned_to': dtype('O'),
 'knowledge': dtype('bool'),
 'u_priority_confirmation': dtype('bool'),
 'notify': dtype('O'),
 'problem_id': dtype('O'),
 'rfc': dtype('O'),
 'vendor': dtype('O'),
 'caused_by': dtype('O'),
 'closed_code': dtype('O'),
 'resolved_by': dtype('O'),
 'resolved_at': 'datetime64[ns]',
 'closed_at': 'd

In [30]:
ddf.dtypes

number                             object
incident_state                     object
active                             object
reassignment_count                  int64
reopen_count                        int64
sys_mod_count                       int64
made_sla                           object
caller_id                          object
opened_by                          object
opened_at                  datetime64[ns]
sys_created_by                     object
sys_created_at             datetime64[ns]
sys_updated_by                     object
sys_updated_at             datetime64[ns]
contact_type                       object
location                           object
category                           object
subcategory                        object
u_symptom                          object
cmdb_ci                            object
impact                             object
urgency                            object
priority                           object
assignment_group                  

In [31]:
ddf_qy = ddf[(ddf["closed_at"].dt.quarter == 2) & (ddf["closed_at"].dt.year == 2016)]
fp = "../data/ticket_resolution_Q2_2016.csv"
ddf_qy.to_csv(fp, index=False)


In [32]:
df_types = ddf.dtypes.to_frame().reset_index()
df_types.columns = ["attribute", "type"]
fp ="../data/data_types.csv"
df_types.to_csv(fp, index=False)

In [33]:
df_types

,attribute,type
0,number,object
1,incident_state,object
2,active,object
3,reassignment_count,int64
4,reopen_count,int64
5,sys_mod_count,int64
6,made_sla,object
7,caller_id,object
8,opened_by,object
9,opened_at,datetime64[ns]
